# Подготовка данных для ASR

## Импорты

In [63]:
import re

import pandas as pd
from torch.utils.data import Dataset

from dataset import IPS1ASRDataset

## Датасеты

In [51]:
ips_dataset_train = IPS1ASRDataset('../tatar_tts/train/')
ips_dataset_valid = IPS1ASRDataset('../tatar_tts/valid/')

In [52]:
def from_dataset_to_df(dataset: Dataset) -> pd.DataFrame:
    data = []
    for index in range(len(dataset)):
        item = dataset.get_metadata(index)
        item_dict = {
            # 'id': item[0].split('/')[-1][:-4],
            'name': item[0].split('/')[-1],
            # 'path': item[0],
            'text': item[2].lower()
        }
        data.append(item_dict)
    df = pd.DataFrame(data)
    del data
    del item_dict
    del item
    return df

In [53]:
df_train = from_dataset_to_df(ips_dataset_train)
df_valid = from_dataset_to_df(ips_dataset_valid)

In [54]:
df_train.head()

,name,text
0,433.318.wav,караса аны җен шулай чакыра ди
1,348.39.wav,"без­нең бер генә кораб та зарарланмый,"
2,402.165.wav,"чистартып көннән-көн, сәгатьтән-сәгатькә көтә..."
3,418.88.wav,ни өчен русьта агач архитектура өстенлек иткән
4,423.187.wav,малайның уйлары шул урында өзелде


In [55]:
df_valid.head()

,name,text
0,282.311.wav,﻿бүген
1,20.2.wav,"﻿ниһаять, халисә үзен кулга алды, плащын һәм ш..."
2,272.354.wav,﻿бөгелеп төшмәве
3,270.262.wav,﻿бар уңышка ирешкән командалар
4,276.116.wav,﻿ураза кешенең сәламәтлегенә зыян салса шулай ...


## Исследование данных

### Цифры

In [77]:
def get_digits(text):
    numbers = re.findall(r'\d+', text)
    return numbers == []

In [83]:
assert get_digits('бөгелеп төшмәве') == True, 'Неверная работа функции get_digits'
assert get_digits('бөгелеп төшмәве 1') == False, 'Неверная работа функции get_digits'
assert get_digits('1 бөгелеп төшмәве 2') == False, 'Неверная работа функции get_digits'

In [84]:
df_train['numbers'] = df_train['text'].apply(lambda row: get_digits(row))
df_valid['numbers'] = df_valid['text'].apply(lambda row: get_digits(row))

In [89]:
df_train.loc[df_train['numbers'] == False].head()

,name,text,numbers
1406,491.336.wav,җанисәп буенча күпчелекне алып торучы халыклар...,False
2301,496.246.wav,тдгпуның «мизгел» яшьләр театры 19 февральдә б...,False
3685,470.130.wav,2010 ел җанисәбен алу буенча биредә 3 кешенең ...,False
5055,480.221.wav,беренче төркем районнарга инде 9 апрельдә үк к...,False
5444,482.237.wav,тр дәүләт советы секретаре лилия маврина белде...,False


In [90]:
df_valid.loc[df_valid['numbers'] == False].head()

,name,text,numbers


In [88]:
df_train.loc[df_train['numbers'] == False].shape[0]

33

Найдено 33 строки в тренировочном наборе данных, в которых встречаюся числительные

### Специальные символы

In [115]:
def get_char_dijits(text):
    special_characters = '@#$%^&*()-+_=<>/\'":;[]{}\\|~`'
    for char in text:
        if char in special_characters:
            return False
    return True

In [116]:
assert get_char_dijits('бөгелеп төшмәве') == True, 'Неверная работа функции get_digits'
assert get_char_dijits('бөгелеп төшмәве ""') == False, 'Неверная работа функции get_digits'
assert get_char_dijits('1 бөгелеп төшмәве ..\\||') == False, 'Неверная работа функции get_digits'

In [117]:
df_train['char_dijits'] = df_train['text'].apply(lambda row: get_char_dijits(row))
df_valid['char_dijits'] = df_valid['text'].apply(lambda row: get_char_dijits(row))

In [118]:
df_train.loc[df_train['char_dijits'] == False].head()

,name,text,numbers,char_dijits
2,402.165.wav,"чистартып көннән-көн, сәгатьтән-сәгатькә көтә...",True,False
14,350.171.wav,-нәрсә җыйнап алырга? - дип сораган булды.,True,False
16,109.13.wav,﻿ә шулай да җаны тыныч түгел; җаны һаман нидер...,True,False
27,350.70.wav,соңгы елларда безнең авылларыбыз көннән-көн ма...,True,False
41,387.45.wav,ай-сезынг-юган русия территориясеннән агучы елга,True,False


In [119]:
df_train.loc[df_train['char_dijits'] == False].shape[0]

7784

In [120]:
df_valid.loc[df_valid['char_dijits'] == False].head()

,name,text,numbers,char_dijits
39,216.7.wav,﻿як-яктан бертуктаусыз сораулар ява тора.,True,False
48,279.405.wav,﻿белеп-аңлап җитә алмассың,True,False
52,211.12.wav,﻿мең спортчының биш-алтысы гына финишка барып ...,True,False
54,232.8.wav,"﻿хатыннар чибәр, затлы киенгәннәр, берсеннән-б...",True,False
76,281.273.wav,﻿- кая үтәсез,True,False


In [121]:
df_valid.loc[df_train['char_dijits'] == False].shape[0]

1515

Мы разрешаем иметь в данных !?, символы. остальные нужно будет удалить из датасета